In [37]:
# 20200612 JLCY

from netCDF4 import Dataset
import glob
# import wrf
import xarray as xr
import datetime
import numpy as np

## extract WRF data (from Eagle)

In [9]:
target_height = [ 40.,  50.,  60.,  80., 100., 120., 140., 160., 180., 200.] # agl

lat_start, lat_end = 410, 560
lon_start, lon_end = 390, 510

wrf_target_dir = '/Users/jlee/eagle/wfip2_wrf_from_eagle/'
output_dir = '/Users/jlee/eagle/wfip2_wrf_py/'

wrf_file_list = glob.glob(wrf_target_dir+'*')
wrf_file_list.sort()

print(wrf_file_list[0])

/Users/jlee/eagle/wfip2_wrf_from_eagle/wrfout_d02_2016-09-23_12:00:00


In [8]:
for file in wrf_file_list: 

    wrf_data = Dataset(file, 'r')

    ua = wrf.getvar(wrf_data, 'ua')
    va = wrf.getvar(wrf_data, 'va')
    ght = wrf.g_geoht.get_height(wrf_data, msl=False)

    ua_subset = ua[:, lat_start:lat_end, lon_start:lon_end]
    va_subset = va[:, lat_start:lat_end, lon_start:lon_end]
    ght_subset = ght[:, lat_start:lat_end, lon_start:lon_end]

    ua_ght = wrf.interplevel(ua_subset, ght_subset, target_height)
    va_ght = wrf.interplevel(va_subset, ght_subset, target_height)

    del ua_ght.attrs['projection']
    del va_ght.attrs['projection']

    ds = xr.Dataset({'u': ua_ght, 'v': va_ght})
    ds.to_netcdf(output_dir+file.split('/')[-1]+'.nc')

## extract WRF subset data (local)

In [48]:
wrf_sub_dir = '/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/'
wrf_sub_file = 'wrfout_d02_2016-09-23_12:00:00.nc'
wrf_sub_data = Dataset(wrf_sub_dir+wrf_sub_file, 'r')

In [49]:
wrf_sub_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): south_north(150), west_east(120), level(10)
    variables(dimensions): float32 XLONG(south_north,west_east), float32 XLAT(south_north,west_east), float32 XTIME(), int64 Time(), float64 level(level), float32 u(level,south_north,west_east), float32 v(level,south_north,west_east)
    groups: 

In [50]:
np.where(wrf_sub_data['level'][:].data == 120)[0][0]

5

In [51]:
wrf_sub_data['XLAT'][71][40]

45.57153

In [116]:
wrf_sub_data['XLONG'][71][40]

-120.747986

In [117]:
wrf_sub_data['u'][3, 71, 40]

masked_array(data=2.5444298,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [118]:
wrf_sub_data['v'][3, 71, 40]

masked_array(data=1.308059,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [80]:
s = wrf_sub_file.split('_')[2]+'_'+wrf_sub_file.split('_')[3].split('.')[0]
t = datetime.datetime.strptime(s, "%Y-%m-%d_%H:%M:%S")

In [81]:
t

datetime.datetime(2016, 9, 23, 12, 0)

In [84]:
t + datetime.timedelta(seconds=3000)

datetime.datetime(2016, 9, 23, 12, 50)

In [49]:
wrf_sub_data['u']

<class 'netCDF4._netCDF4.Variable'>
float32 u(level, south_north, west_east)
    _FillValue: 9.96921e+36
    FieldType: 104
    units: m s-1
    stagger: 
    coordinates: XLONG XTIME XLAT Time
    missing_value: 9.969209968386869e+36
    vert_units: m
unlimited dimensions: 
current shape = (10, 150, 120)
filling on

In [54]:
wrf_sub_data['level'][3]

masked_array(data=80.,
             mask=False,
       fill_value=1e+20)

## trim down wrf output for GitHub

In [6]:
lat_start, lat_end = 68, 74
lon_start, lon_end = 37, 43

wrf_target_dir = '/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/'
output_dir = '/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/trim/'

wrf_file_list = glob.glob(wrf_target_dir+'*.nc')
# wrf_file_list = [wrf_target_dir+'wrfout_d02_2016-09-23_12:00:00.nc', 
#                  wrf_target_dir+'wrfout_d02_2016-09-23_12:05:00.nc']
wrf_file_list.sort()

for file in wrf_file_list: 
    
    print(file)

    wrf_data = Dataset(file, 'r')
    
    #print(wrf_sub_data)
    
    xlat = wrf_data['XLAT'][lat_start:lat_end, lon_start:lon_end]
    xlon = wrf_data['XLONG'][lat_start:lat_end, lon_start:lon_end]
    xlevel = wrf_data['level'][:]

    u_subset = wrf_data['u'][:, lat_start:lat_end, lon_start:lon_end]
    v_subset = wrf_data['v'][:, lat_start:lat_end, lon_start:lon_end]
       
    newfile = Dataset((output_dir+file.split('/')[-1]), mode='w', format='NETCDF4_CLASSIC') 
    
    lat_dim = newfile.createDimension('south_north', lat_end-lat_start)     # latitude axis
    lon_dim = newfile.createDimension('west_east', lon_end-lon_start)    # longitude axis
    lev_dim = newfile.createDimension('level', len(xlevel)) # unlimited axis (can be appended to).
    
    lat = newfile.createVariable('XLAT', np.float32, ('south_north', 'west_east'))
    lon = newfile.createVariable('XLONG', np.float32, ('south_north', 'west_east'))
    level = newfile.createVariable('level', np.float64, ('level',))
    u = newfile.createVariable('u', np.float32, ('level', 'south_north', 'west_east'))
    v = newfile.createVariable('v', np.float32, ('level', 'south_north', 'west_east'))
    
    lat[:,:] = xlat
    lon[:,:] = xlon
    level[:] = xlevel
    
    u[:,:,:] = u_subset
    v[:,:,:] = v_subset
    
    #ds = xr.Dataset({'u': u_subset, 'v': v_subset})
    #ds.to_netcdf(output_dir+file.split('/')[-1]+'.nc')
    
    newfile.close()

/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:05:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:10:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:15:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:20:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:25:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:40:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12:45:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_12

7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:25:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:40:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:45:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:50:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_19:55:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_20:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_20:05:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-23_20:10:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_

7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_02:50:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_02:55:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:05:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:10:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:15:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:20:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:25:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_03:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_

7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:15:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:20:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:25:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:40:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:45:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:50:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_10:55:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_11:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_

7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_17:55:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:05:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:10:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:15:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:20:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:25:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-24_18:40:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_

7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_01:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_01:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_01:40:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_01:45:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_01:50:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_01:55:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_02:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_02:05:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_02:10:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_02:15:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_

7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:00:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:05:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:10:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:15:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:20:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:25:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:30:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:35:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:40:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_2016-09-25_09:45:00.nc
7
(10, 6, 6)
/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/wrfout_d02_

In [55]:
wrf_sub_dir = '/Users/jlee/eagle/wfip2_wrf_subset_from_eagle/trim/'
wrf_sub_file = 'wrfout_d02_2016-09-23_12:00:00.nc'
wrf_sub_data = Dataset(wrf_sub_dir+wrf_sub_file, 'r')
wrf_sub_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): lat(6), lon(6), level(10)
    variables(dimensions): float32 lat(lat,lon), float32 lon(lon,lon), float64 level(level), float32 u(level,lat,lon), float32 v(level,lat,lon)
    groups: 

In [56]:
wrf_sub_data['lat'][3][3]

45.57153

In [57]:
wrf_sub_data['u'][3, 3, 3]

masked_array(data=2.5444298,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

## examine ex1 data

In [4]:
wrf_ex1_dir = '/Users/jlee/Documents/GitHub/nwtc-ivalidate/data/ex1/'
wrf_ex1_file = 'cut_wrfout_d01_2016-03-07_09_00_00'
wrf_ex1_data = Dataset(wrf_ex1_dir+wrf_ex1_file, 'r')

In [5]:
wrf_ex1_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    TITLE:  OUTPUT FROM WRF V3.7+ MODEL
    START_DATE: 2016-03-07_06:00:00
    SIMULATION_START_DATE: 2016-03-07_06:00:00
    WEST-EAST_GRID_DIMENSION: 1800
    SOUTH-NORTH_GRID_DIMENSION: 1060
    BOTTOM-TOP_GRID_DIMENSION: 51
    DX: 3000.0
    DY: 3000.0
    SKEBS_ON: 0
    SPEC_BDY_FINAL_MU: 0
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 1
    KM_OPT: 4
    DAMP_OPT: 3
    DAMPCOEF: 0.2
    KHDIF: 0.0
    KVDIF: 0.0
    MP_PHYSICS: 28
    RA_LW_PHYSICS: 4
    RA_SW_PHYSICS: 4
    SF_SFCLAY_PHYSICS: 5
    SF_SURFACE_PHYSICS: 3
    BL_PBL_PHYSICS: 5
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 1
    SST_UPDATE: 0
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0
    SGFDDA_INTERVAL_M: 0
    SGFDDA_END_H: 0
    HYPSOMETRIC_OPT: 2
    USE_THETA_M: 0
    SF_URBAN_PHYSICS: 0
    SHCU_PHYSICS: 0
    MFSHCONV: 0
    FEEDBACK: 0
    

In [13]:
wrf_ex1_data['Times'][0]

masked_array(data=[b'2', b'0', b'1', b'6', b'-', b'0', b'3', b'-', b'0',
                   b'7', b'_', b'0', b'9', b':', b'0', b'0', b':', b'0',
                   b'0'],
             mask=False,
       fill_value=b'N/A',
            dtype='|S1')

In [50]:
wrf_ex1_data['WSPD80']

<class 'netCDF4._netCDF4.Variable'>
float32 WSPD80(Time, south_north, west_east)
    FieldType: 104
    MemoryOrder: XY 
    description: Wind Speed at 80 M
    units: m s-1
    stagger: 
    coordinates: XLONG XLAT XTIME
unlimited dimensions: Time
current shape = (1, 1059, 1799)
filling on, default _FillValue of 9.969209968386869e+36 used

## SODAR data

In [119]:
sodar_target_dir = '/Users/jlee/eagle/wfip2_sodar_from_eagle/'

# 10-minute files
sodar_file_list = glob.glob(sodar_target_dir+'*')
sodar_file_list.sort()

n = 72

sodar_data = Dataset(sodar_file_list[n], 'r')
print(sodar_file_list[n])

/Users/jlee/eagle/wfip2_sodar_from_eagle/sodar.z06.b0.20160923.120000.txt.a2e.nc


In [127]:
sodar_data_check = Dataset(sodar_target_dir+'sodar.z06.b0.20160923.223000.txt.a2e.nc', 'r')

In [131]:
sodar_data_check['wind_speed'][0,3]

masked_array(data=5.44,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [107]:
sodar_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2017-04-13 05:45:25 created by libingest-1.0 using wfip2_sodar-1.0 (build version: v1.1.0)
    dimensions(sizes): time(1), height(10), bounds(2)
    variables(dimensions): float64 time(time), float64 time_bounds(time,bounds), float32 height(height), float32 wind_speed(time,height), float32 wind_direction(time,height), float32 vertical_air_velocity(time,height), float64 latitude(), float64 longitude(), float64 altitude()
    groups: 

In [120]:
sodar_data['time']

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time offset from midnight
    standard_name: time
    units: seconds since 2016-09-23 00:00:00 0:00
    bounds: time_bounds
unlimited dimensions: time
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [121]:
sodar_data['time'].units

'seconds since 2016-09-23 00:00:00 0:00'

In [122]:
'_'.join(sodar_data['time'].units.split(' ')[2:4])

'2016-09-23_00:00:00'

In [123]:
sodar_data['time'][0]

masked_array(data=43200.,
             mask=False,
       fill_value=1e+20)

In [105]:
sodar_data['time_bounds']

<class 'netCDF4._netCDF4.Variable'>
float64 time_bounds(time, bounds)
unlimited dimensions: time
current shape = (1, 2)
filling on, default _FillValue of 9.969209968386869e+36 used

In [126]:
sodar_data['wind_speed'][0, 3]

masked_array(data=8.94,
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [50]:
sodar_data['latitude'][0]

masked_array(data=45.57451,
             mask=False,
       fill_value=1e+20)

In [51]:
sodar_data['longitude'][0]

masked_array(data=-120.74734,
             mask=False,
       fill_value=1e+20)

In [52]:
sodar_data['time'][0]

masked_array(data=0.,
             mask=False,
       fill_value=1e+20)

In [63]:
sodar_data['wind_speed'][:].data

array([[2.82000e+00, 2.93000e+00, 2.88000e+00, 2.48000e+00, 3.21000e+00,
        2.74000e+00, 3.11000e+00, 3.36000e+00, 2.59000e+00, 9.96921e+36]],
      dtype=float32)

In [64]:
sodar_data['wind_speed']

<class 'netCDF4._netCDF4.Variable'>
float32 wind_speed(time, height)
    long_name: wind speed
    standard_name: wind_speed
    units: m/s
    cell_methods: time: mean
unlimited dimensions: time
current shape = (1, 10)
filling on, default _FillValue of 9.969209968386869e+36 used

In [16]:
sodar_data['height'][:].data

array([ 40.,  50.,  60.,  80., 100., 120., 140., 160., 180., 200.],
      dtype=float32)

In [132]:
sodar_data['height'][:].data == 120

array([False, False, False, False, False,  True, False, False, False,
       False])

In [141]:
np.where(sodar_data['height'][:].data == 120)[0][0]

5

In [138]:
sodar_data['height'][:].data[np.where(sodar_data['height'][:].data == 120)]

array([120.], dtype=float32)

## masked data investigation

In [12]:
sodar_dir = '/Users/leec813/GitHub/nwtc-ivalidate/data/mw_data_sodar/'
sodar_data = Dataset(sodar_dir+'sodar.z06.b0.20160923.160000.txt.a2e.nc', 'r')

In [35]:
sodar_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2017-04-13 05:39:57 created by libingest-1.0 using wfip2_sodar-1.0 (build version: v1.1.0)
    dimensions(sizes): time(1), height(10), bounds(2)
    variables(dimensions): float64 time(time), float64 time_bounds(time, bounds), float32 height(height), float32 wind_speed(time, height), float32 wind_direction(time, height), float32 vertical_air_velocity(time, height), float64 latitude(), float64 longitude(), float64 altitude()
    groups: 

In [14]:
sodar_data['wind_speed'][:]

masked_array(data=[[6.2 , 6.9 , 5.96, 6.99, 7.4 , 7.6 , 6.36, 6.7 , 6.93,
                    6.92]],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [30]:
sodar_dir = '/Users/leec813/GitHub/nwtc-ivalidate/data/mw_data_sodar/'
sodar_data = Dataset(sodar_dir+'sodar.z06.b0.20160923.161000.txt.a2e.nc', 'r')
sodar_data['wind_speed'][0]

masked_array(data=[--, --, --, --, --, --, --, --, --, 4.670000076293945],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True, False],
       fill_value=9.96921e+36,
            dtype=float32)

In [32]:
sodar_data['wind_speed'][0][1]

masked

In [34]:
if np.ma.is_masked(sodar_data['wind_speed'][0][1]) is True: 
    print('hey')

hey
